In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq
from functools import reduce

In [131]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [132]:
conn = sq.connect('../../../Pipeline-Census-Bureau/Outputs/CensusBureau.db')

#2000

#SF12000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2000]', conn)
sf12000 = pd.DataFrame(sql_query)
sf12000.drop(columns = 'Source', inplace = True)
#SF32000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF3_2000]', conn)
sf32000 = pd.DataFrame(sql_query)
sf32000.drop(columns = 'Source', inplace = True)
#TIGER2000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2000]', conn)
TIGER2000 = pd.DataFrame(sql_query)
TIGER2000.drop(columns = 'Source', inplace = True)
dfs2000 = [TIGER2000, sf12000, sf32000]
twothousand= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2000)
twothousand = twothousand.set_index(['NAME', 'GEO_ID'])
twothousandsuff = twothousand.add_suffix(' 2000_1YR')


#2010


#ACS52010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2010]', conn)
ACS52010 = pd.DataFrame(sql_query)
ACS52010.drop(columns = 'Source', inplace = True)
twothousandten = ACS52010.set_index(['NAME', 'GEO_ID'])
twothousandten = twothousandten.drop(columns = 'index')
twothousandtenfivesuff = twothousandten.add_suffix(' 2010_5YR')

#SF12010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2010]', conn)
sf12010 = pd.DataFrame(sql_query)
sf12010.drop(columns = 'Source', inplace = True)
#TIGER2010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2010]', conn)
TIGER2010 = pd.DataFrame(sql_query)
TIGER2010.drop(columns = 'Source', inplace = True)
dfs2010 = [TIGER2010, sf12010]
twothousandten= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2010)
twothousandten = twothousandten.fillna(0)
twothousandten = twothousandten.set_index(['NAME', 'GEO_ID'])
twothousandtensuff = twothousandten.add_suffix(' 2010_1YR')

#"CURRENT"
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#TIGER2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2020]', conn)
TIGER2020 = pd.DataFrame(sql_query)
TIGER2020.drop(columns = 'Source', inplace = True)
dfs2020 = [TIGER2020, PL2020]
twothousandtwenty = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2020)
twothousandtwenty = twothousandtwenty.fillna(0)
twothousandtwenty = twothousandtwenty.set_index(['NAME', 'GEO_ID'])
twothousandtwentysuff = twothousandtwenty.add_suffix(' 2020_1YR')
#ACS52022
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_1]', conn)
ACS52022_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_2]', conn)
ACS52022_2 = pd.DataFrame(sql_query)
ACS52022 = ACS52022_1.merge(ACS52022_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52022.drop(columns = 'Source', inplace = True)
ACS52022 = ACS52022.fillna(0)
ACS52022 = ACS52022.set_index(['NAME', 'GEO_ID'])
twothousandtwentytwosuff = ACS52022.add_suffix(' 2022_5YR')

#PEP Population and Housing Units
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PEP_PopHU_20102023]', conn)
pep = pd.DataFrame(sql_query)
hu2020 = pep.loc[pep['Year'] == '2020']
hu2020 = hu2020.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population', 'Population % Change', 
                                'Population Change', 'Year'])
hu2020 = hu2020.fillna(0)
hu2020 = hu2020.set_index(['NAME', 'GEO_ID'])
hu2020suff = hu2020.add_suffix(' 2020_PEP')
hu2010 = pep.loc[pep['Year'] == '2010']
hu2010 = hu2010.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population', 'Population % Change', 
                                'Population Change', 'Year'])
hu2010 = hu2010.fillna(0)
hu2010 = hu2010.set_index(['NAME', 'GEO_ID'])
hu2010suff = hu2010.add_suffix(' 2010_PEP')

pep23 = pep.loc[pep['Year'] == '2023']
pep23 = pep23.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population % Change', 'Population Change', 'Year'])
pep23 = pep23.fillna(0)
pep23 = pep23.set_index(['NAME', 'GEO_ID'])
pepsuff23 = pep23.add_suffix(' 2023_PEP')

pep20 = pep.loc[pep['Year'] == '2020']
pep20 = pep20.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population % Change', 'Population Change', 'Year'])
pep20 = pep20.fillna(0)
pep20 = pep20.set_index(['NAME', 'GEO_ID'])
pepsuff20 = pep20.add_suffix(' 2020_PEP')

pep10 = pep.loc[pep['Year'] == '2010']
pep10 = pep10.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population % Change', 'Population Change', 'Year'])
pep10 = pep10.fillna(0)
pep10 = pep10.set_index(['NAME', 'GEO_ID'])
pepsuff10 = pep10.add_suffix(' 2010_PEP')

In [133]:
geos = ['Wilson County, Tennessee']

In [134]:
dfs = [twothousandsuff, twothousandtensuff, twothousandtenfivesuff, twothousandtwentysuff, twothousandtwentytwosuff, pepsuff20, 
       pepsuff23, pepsuff10, hu2020suff, hu2010suff]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'], how='outer'), dfs)

In [135]:
dfsuff = dfsuff.reset_index().set_index('NAME').transpose()
#dfsuff = dfsuff.rename(columns = namestocommon)
dfsuff = dfsuff[geos]
dfsuff = dfsuff.transpose().reset_index()

In [136]:
dfsuff.head()

,NAME,GEO_ID,ALAND 2000_1YR,Square Acres 2000_1YR,Square Miles 2000_1YR,Population 2000_1YR,Male Under 5 2000_1YR,Female Under 5 2000_1YR,Male 5 to 14 2000_1YR,Female 5 to 14 2000_1YR,Male 15 to 24 2000_1YR,Female 15 to 24 2000_1YR,Male 5 to 9 2000_1YR,Female 5 to 9 2000_1YR,Male 10 to 14 2000_1YR,Female 10 to 14 2000_1YR,Male 15 to 17 2000_1YR,Female 15 to 17 2000_1YR,Male 18 to 24 2000_1YR,Female 18 to 24 2000_1YR,Male 25 to 34 2000_1YR,Female 25 to 34 2000_1YR,Male 35 to 44 2000_1YR,Female 35 to 44 2000_1YR,Male 45 to 54 2000_1YR,Female 45 to 54 2000_1YR,Male 55 to 64 2000_1YR,Female 55 to 64 2000_1YR,Male 65 to 74 2000_1YR,Female 65 to 74 2000_1YR,Male 75 to 84 2000_1YR,Female 75 to 84 2000_1YR,Male 85 and Older 2000_1YR,Female 85 and Older 2000_1YR,Age:Under 5 2000_1YR,Age %:Under 5 2000_1YR,Age:5 to 9 2000_1YR,Age%:5 to 9 2000_1YR,Age:10 to 14 2000_1YR,Age%:10 to 14 2000_1YR,Age:15 to 17 2000_1YR,Age%:15 to 17 2000_1YR,Age:18 to 24 2000_1YR,Age%:18 to 24 2000_1YR,Age:25 to 34 2000_1YR,Age%:25 to 34 2000_1YR,Age:35 to 44 2000_1YR,Age%:35 to 44 2000_1YR,Age:45 to 54 2000_1YR,Age%:45 to 54 2000_1YR,Age:55 to 64 2000_1YR,Age%:55 to 64 2000_1YR,Age:65 to 74 2000_1YR,Age%:65 to 74 2000_1YR,Age:75 to 84 2000_1YR,Age%:75 to 84 2000_1YR,Age:85 and Older 2000_1YR,Age%:85 and Older 2000_1YR,Age:School Age 5 to 18 2000_1YR,Age%:School Age 5 to 18 2000_1YR,Age:Under 18 2000_1YR,Age%:Under 18 2000_1YR,Age:18 to 54 2000_1YR,Age%:18 to 54 2000_1YR,Age:18 to 64 2000_1YR,Age%:18 to 64 2000_1YR,Age:55 and Older 2000_1YR,Age%:55 and Older 2000_1YR,Age:65 and Older 2000_1YR,Age%:65 and Older 2000_1YR,White Alone 2000_1YR,White Alone % 2000_1YR,Non-White Population: Race 2000_1YR,Black or African American Alone 2000_1YR,Black or African American Alone % 2000_1YR,Black or African American Alone % Non-White Population 2000_1YR,American Indian Alaska Native Alone 2000_1YR,American Indian Alaska Native Alone % 2000_1YR,American Indian Alaska Native Alone % Non-White Population 2000_1YR,Asian Alone 2000_1YR,Asian Alone % 2000_1YR,Asian Alone % Non-White Population 2000_1YR,Native Hawaiian Other Pacific Islander Alone 2000_1YR,Native Hawaiian Other Pacific Islander Alone % 2000_1YR,Native Hawaiian Other Pacific Islander Alone % Non-White Population 2000_1YR,Some Other Race Alone 2000_1YR,Some Other Race Alone % 2000_1YR,Some Other Race Alone % Non-White Population 2000_1YR,Two or More Races 2000_1YR,Two or More Races % 2000_1YR,Two or More Races % Non-White Population 2000_1YR,Hispanic or Latino 2000_1YR,Hispanic or Latino % 2000_1YR,Not Hispanic or Latino 2000_1YR,Not Hispanic or Latino % 2000_1YR,Minority 2000_1YR,Minority % 2000_1YR,White Alone Not Hispanic or Latino 2000_1YR,White Alone Not Hispanic or Latino % 2000_1YR,Total Households 2000_1YR,Family Households 2000_1YR,Family Households: Married Couple Family 2000_1YR,"Household Type%: Family, Married Couple Households 2000_1YR",Family Households: Not Married Couple Family 2000_1YR,"Household Type%: Family, Not Married Couple Households 2000_1YR",Family Households: Not Married Couple: Male no Spouse 2000_1YR,Family Households: Not Married Couple: Female no Spouse 2000_1YR,Nonfamily Households 2000_1YR,Nonfamily Households: Householder Alone 2000_1YR,"Household Type%: Nonfamily, Householder Alone 2000_1YR",Nonfamily Households: Householder not Alone 2000_1YR,"Household Type%: Nonfamily, Householder Not Alone 2000_1YR",Average Household Size 2000_1YR,Occupancy:Total Households 2000_1YR,Occupancy:Occupied Units 2000_1YR,Occupancy%:Occupied Units 2000_1YR,Occupancy:Vacant Units 2000_1YR,Occupancy%:Vacant Units 2000_1YR,Tenure:Total Households 2000_1YR,Tenure:Owners 2000_1YR,Tenure%:Owners 2000_1YR,Tenure:Renters 2000_1YR,Tenure%:Renters 2000_1YR,Median Household Income 2000_1YR,HHIncome:Total Households 2000_1YR,"HHIncome:Less than 10,000 2000_1YR","HHIncome%:Less than 10,000 2000_1YR","HHIncome:10 to 14,999 2000_1YR","HHIncome%:10 to 14,999 2000_1YR","HHIncome:15 to 19,999 2000_1YR","HHIncome%:

Wilson County current population  
1.	Total count   
2.	Percent growth of pop since 2010  
3.	Percent of school-aged children  
4.	Percent living below poverty level  
5.	Percent of senior citizens  
6.	Percent of ethnic minorities  
7.	Average household size  
8.	Percent of Homeowner’s cost burdened  
9.	Percent of Renters cost burdened  
10.	County Median Household income  

Wilson County current employment  
1.	Total employment  
2.	Percent growth in jobs since 2010  
3.	Percent in white collar jobs, and percent in blue collar jobs


In [137]:
cols = ['NAME',
        #most current population
        'Population 2023_PEP',
        #2010 population for change
        'Population 2010_1YR',
        #2020 population for change
        'Population 2020_1YR',
        #% school aged children
        'Age%:School Age 5 to 18 2022_5YR',
        #% below poverty level
        'Poverty%: Below Poverty Level 2022_5YR', 
        #% senior citizens
        'Age%:65 and Older 2022_5YR', 
        #ethnic minorities
        'Minority % ACS 2022_5YR', 
        #average household size
        'Average Household Size 2022_5YR', 
        #% homeowners cost burdened
        'CB%: Cost Burdened Homeowners 2022_5YR', 
        #% renters cost burdened
        'CB%:Cost Burdened Renters 2022_5YR', 
        #median household income
        'Median Household Income 2022_5YR'
       ]
df = dfsuff[cols]

In [138]:
df['Population % Increase 2010 to 2020 1 YR EST'] = percentchange(df['Population 2020_1YR'], df['Population 2010_1YR'])

In [139]:
dem = df

## Jobs

In [140]:
conn = sq.connect('../../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')

sql_query = pd.read_sql('SELECT * FROM [WP2024_IndustryEmployment_Annual_Change_UnadjustedNoReplacements]', conn)
data = pd.DataFrame(sql_query)
data = data.loc[data['NAME'] == 'Wilson County, Tennessee']
data = data.loc[data['Time Frame'] == 'None']

In [141]:
data10 = data.loc[data['Year'] == '2010']
data10 = data10[['NAME', 'Ind:Total']]
data10 = data10.set_index(['NAME'])
data10 = data10.add_suffix(' 2010_BEA')

data20 = data.loc[data['Year'] == '2020']
data20 = data20[['NAME', 'Ind:Total']]
data20 = data20.set_index(['NAME'])
data20 = data20.add_suffix(' 2020_BEA')

data23 = data.loc[data['Year'] == '2022']
data23 = data23[['NAME', 'Ind:Total', 'Ind%:Snapshot: White Collar', 'Ind%:Snapshot: Blue Collar']]
data23 = data23.set_index(['NAME'])
data23 = data23.add_suffix(' 2022_BEA')

In [142]:
dfs = [data10, data20, data23]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME'], how='outer'), dfs)

In [143]:
dfsuff['Employment % Increase 2010 to 2020'] = percentchange(dfsuff['Ind:Total 2020_BEA'], dfsuff['Ind:Total 2010_BEA'])

In [144]:
dfsuff.head()

,Ind:Total 2010_BEA,Ind:Total 2020_BEA,Ind:Total 2022_BEA,Ind%:Snapshot: White Collar 2022_BEA,Ind%:Snapshot: Blue Collar 2022_BEA,Employment % Increase 2010 to 2020
NAME,,,,,,
"Wilson County, Tennessee",53289.0,79255.0,94211.0,34.470497,34.957701,48.726754


In [145]:
ec = dfsuff

In [146]:
df = dem.merge(ec, on = 'NAME')

In [147]:
df.head()

,NAME,Population 2023_PEP,Population 2010_1YR,Population 2020_1YR,Age%:School Age 5 to 18 2022_5YR,Poverty%: Below Poverty Level 2022_5YR,Age%:65 and Older 2022_5YR,Minority % ACS 2022_5YR,Average Household Size 2022_5YR,CB%: Cost Burdened Homeowners 2022_5YR,CB%:Cost Burdened Renters 2022_5YR,Median Household Income 2022_5YR,Population % Increase 2010 to 2020 1 YR EST,Ind:Total 2010_BEA,Ind:Total 2020_BEA,Ind:Total 2022_BEA,Ind%:Snapshot: White Collar 2022_BEA,Ind%:Snapshot: Blue Collar 2022_BEA,Employment % Increase 2010 to 2020
0,"Wilson County, Tennessee",163674.0,113993.0,147737.0,17.42032,8.290593,15.772388,17.89317,2.68,17.958029,44.51572,89462.0,29.601818,53289.0,79255.0,94211.0,34.470497,34.957701,48.726754


In [148]:
df.to_csv('flyerthing.csv', index = False)